In [4]:
import pandas as pd

# Load the data
df = pd.read_excel('online_retail_II.xlsx')

# See the first 5 rows
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'online_retail_II.xlsx'

In [5]:
import os
print(os.getcwd())

C:\Users\Manoj kumar\OneDrive\Desktop\rfm_project


In [6]:
import os
print(os.listdir())

['.ipynb_checkpoints', 'online_retail_II.csv', 'Untitled.ipynb']


In [13]:
import pandas as pd
df = pd.read_csv('online_retail_II.csv', encoding='unicode_escape')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [14]:
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nMissing Values:")
print(df.isnull().sum())

Shape: (1067371, 8)

Columns: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']

Missing Values:
Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64


In [15]:
print("Before cleaning:", len(df), "rows")

# Remove rows with no Customer ID
df = df.dropna(subset=['Customer ID'])
print("After removing missing customers:", len(df), "rows")

# Remove cancelled orders (invoices starting with C)
df = df[~df['Invoice'].astype(str).str.startswith('C')]
print("After removing cancellations:", len(df), "rows")

# Remove negative or zero quantities and prices
df = df[df['Quantity'] > 0]
df = df[df['Price'] > 0]
print("After removing bad data:", len(df), "rows")

Before cleaning: 1067371 rows
After removing missing customers: 824364 rows
After removing cancellations: 805620 rows
After removing bad data: 805549 rows


In [16]:
# Create TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['Price']

# Fix date column
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Fix Customer ID
df['Customer ID'] = df['Customer ID'].astype(int)

print("Done! Final shape:", df.shape)
df.head()

Done! Final shape: (805549, 9)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


In [17]:
print("Date range:")
print("From:", df['InvoiceDate'].min())
print("To:", df['InvoiceDate'].max())

print("\nTotal Revenue: £", round(df['TotalPrice'].sum(), 2))
print("Unique Customers:", df['Customer ID'].nunique())
print("\nTop 5 Countries:")
print(df['Country'].value_counts().head())

Date range:
From: 2009-12-01 07:45:00
To: 2011-12-09 12:50:00

Total Revenue: £ 17743429.18
Unique Customers: 5878

Top 5 Countries:
Country
United Kingdom    725250
Germany            16694
EIRE               15743
France             13812
Netherlands         5088
Name: count, dtype: int64


In [18]:
# We calculate recency FROM this date
# Using the last date in our dataset as reference
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
print("Reference date:", snapshot_date)

Reference date: 2011-12-10 12:50:00


In [19]:
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'Invoice': 'count',
    'TotalPrice': 'sum'
}).reset_index()

# Rename columns to make it clear
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

print("RFM Table created!")
print(rfm.head(10))

RFM Table created!
   CustomerID  Recency  Frequency  Monetary
0       12346      326         34  77556.46
1       12347        2        253   5633.32
2       12348       75         51   2019.40
3       12349       19        175   4428.69
4       12350      310         17    334.40
5       12351      375         21    300.93
6       12352       36        103   2849.84
7       12353      204         24    406.76
8       12354      232         58   1079.40
9       12355      214         35    947.61


In [20]:
# Recency — lower days = better = higher score
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])

# Frequency — higher count = better = higher score
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 
                          5, labels=[1,2,3,4,5])

# Monetary — higher spend = better = higher score
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

print("Scores added!")
print(rfm.head())

Scores added!
   CustomerID  Recency  Frequency  Monetary R_Score F_Score M_Score
0       12346      326         34  77556.46       2       2       5
1       12347        2        253   5633.32       5       5       5
2       12348       75         51   2019.40       3       3       4
3       12349       19        175   4428.69       5       4       5
4       12350      310         17    334.40       2       2       2


In [21]:
def assign_segment(row):
    r = int(row['R_Score'])
    f = int(row['F_Score'])
    m = int(row['M_Score'])
    
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champion'
    elif r >= 3 and f >= 3:
        return 'Loyal Customer'
    elif r >= 4 and f <= 2:
        return 'Recent User'
    elif r <= 2 and f >= 3:
        return 'At Risk'
    elif r == 1 and f == 1:
        return 'Lost'
    else:
        return 'Needs Attention'

rfm['Segment'] = rfm.apply(assign_segment, axis=1)

print("Segments assigned!")
print(rfm['Segment'].value_counts())


Segments assigned!
Segment
Needs Attention    1394
Loyal Customer     1344
Champion           1273
At Risk             910
Recent User         486
Lost                471
Name: count, dtype: int64


In [22]:
rfm.to_csv('rfm_output.csv', index=False)
print("File saved as rfm_output.csv ")
print("Total customers analyzed:", len(rfm))

File saved as rfm_output.csv 
Total customers analyzed: 5878


In [23]:
#Market Basket Analysis. This finds patterns like "Customers who bought X also bought Y"

In [24]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

print("Libraries loaded ")

Libraries loaded 


In [25]:
# Using only UK data to keep it fast
uk_data = df[df['Country'] == 'United Kingdom']
print("UK transactions:", len(uk_data))

UK transactions: 725250


In [30]:
basket = uk_data.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0)

# Use .map instead of .applymap (newer pandas version)
basket = basket.map(lambda x: 1 if x > 0 else 0)

print("Basket created!")
print("Shape:", basket.shape)

Basket created!
Shape: (33541, 5249)


In [32]:
frequent_items = apriori(basket, min_support=0.02, use_colnames=True)

print("Frequent items found:", len(frequent_items))

C:\Users\Manoj kumar\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:175: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Frequent items found: 180


In [33]:
rules = association_rules(frequent_items, 
                          metric='lift', 
                          min_threshold=1.2)

# Sort by lift (highest = strongest relationship)
rules = rules.sort_values('lift', ascending=False)

print("Rules found:", len(rules))
print("\nTop 5 Product Combinations:")
print(rules[['antecedents','consequents','lift']].head())

Rules found: 18

Top 5 Product Combinations:
                            antecedents                          consequents  \
15     (SWEETHEART CERAMIC TRINKET BOX)     (STRAWBERRY CERAMIC TRINKET BOX)   
14     (STRAWBERRY CERAMIC TRINKET BOX)     (SWEETHEART CERAMIC TRINKET BOX)   
17        (WOODEN FRAME ANTIQUE WHITE )  (WOODEN PICTURE FRAME WHITE FINISH)   
16  (WOODEN PICTURE FRAME WHITE FINISH)        (WOODEN FRAME ANTIQUE WHITE )   
2            (LOVE BUILDING BLOCK WORD)           (HOME BUILDING BLOCK WORD)   

         lift  
15  13.949822  
14  13.949822  
17  11.772558  
16  11.772558  
2   10.023487  


In [34]:
rules.to_csv('basket_rules.csv', index=False)
print("Saved as basket_rules.csv ")

Saved as basket_rules.csv 


In [36]:
df.to_csv('cleaned_sales.csv', index=False)
print("Saved as cleaned_sales.csv")

# Check all 3 files are ready
import os
files = os.listdir()
for f in files:
    if f.endswith('.csv'):
        print("Ready:", f)


Saved as cleaned_sales.csv
Ready: basket_rules.csv
Ready: cleaned_sales.csv
Ready: online_retail_II.csv
Ready: rfm_output.csv


In [37]:
import os
files = os.listdir()
for f in files:
    print(f)

.ipynb_checkpoints
basket_rules.csv
cleaned_sales.csv
online_retail_II.csv
rfm_analysis.ipynb
rfm_output.csv
